In [1]:
## DO NOT RUN THIS.  It is used to build an updated BkAvgAst... table which is already in the processing zip file.
## first step in creating new BkAvgAst... file
## 2017/04/07:  updated for Mar 2017 failure data (Note: only failures before 2017 used since Call data as of Dec-2016)
## 2018/03/08: updated for Dec 2017 failure events (since Call Data only up to Dec17)
## 2018/08/27: updated failure rate details (number fails/number comm bks)
import numpy as np
import pandas as pd
from datetime import datetime
#import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

import os
os.chdir('C:/BankRisk')

df2 = pd.read_csv("df2_2001_202012.csv")
df2['AsOf'] = pd.to_datetime(df2['AsOf'])

In [2]:
fcol = df2.columns[5:]  ## get listing of numeric fields, then...
df2[fcol] = df2[fcol].astype(float)  ### CRITICAL to convert numeric data to FLOAT!!! (since some are ints)

In [3]:
df3 = df2.sort_values(['IDRSSD','AsOf'])

In [4]:
df3.head()

Unnamed: 0  IDRSSD       AsOf                    Name State     Cert  \
0               0      37 2001-03-31  BANK OF HANCOCK COUNTY    GA  10057.0   
8857         8857      37 2001-06-30  BANK OF HANCOCK COUNTY    GA  10057.0   
17647       17647      37 2001-09-30  BANK OF HANCOCK COUNTY    GA  10057.0   
26405       26405      37 2001-12-31  BANK OF HANCOCK COUNTY    GA  10057.0   
35094       35094      37 2002-03-31  BANK OF HANCOCK COUNTY    GA  10057.0   

       eTotAst   eTotLn   eALLL  eOREO  ...  ncoFIfor  ncoAg  ncoCIdom  \
0      65486.0  40507.0   979.0  200.0  ...       0.0    0.0       0.0   
8857   65427.0  42153.0  1049.0    0.0  ...       0.0    0.0       0.0   
17647  65575.0  44359.0  1070.0   40.0  ...       0.0    0.0       0.0   
26405  66624.0  41514.0   987.0   20.0  ...       0.0    0.0       5.0   
35094  68766.0  41539.0  1048.0   20.0  ...       0.0    0.0       0.0   

       ncoCIfor  ncoCard  ncoAuto  ncoOthCsmr  ncoForGovt  ncoOthLn  ncoLeas  
0           0.0      0.0      0.0        -1.0         0.0       0.0      0.0  
8857        0.0      0.0      0.0       -23.0         0.0       0.0      0.0  
17647       0.0      0.0      0.0         2.0         0.0       0.0      0.0  
26405       0.0      0.0      0.0       104.0         0.0       0.0      0.0  
35094       0.0      0.0      0.0       -13.0         0.0       0.0      0.0  

[5 rows x 196 columns]

In [5]:
len(df3) # 481187 (201612), 504444 (201712), 526677(201812), 542745(201909), 568386(202012)

568386

In [6]:
'eDDep' in df3.columns

True

In [7]:
# get mean eTotAst by IDRSSD:
df4 = df3[['IDRSSD','AsOf','Name','State','Cert','eTotAst','aTotAst','eTotLn','eDDep','eFDep']]
df4['eTotDep'] = df4['eDDep'] + df4['eFDep']
df5 = df4[df4['eTotAst']>0].groupby('IDRSSD')

In [8]:
df6x = pd.DataFrame(df5['AsOf'].max()).reset_index()
df6n = pd.DataFrame(df5['AsOf'].min()).reset_index()

In [9]:
df6n.tail()

IDRSSD       AsOf
10793  5472880 2020-12-31
10794  5475452 2020-06-30
10795  5514409 2020-12-31
10796  5518023 2020-09-30
10797  5538937 2020-12-31

In [10]:
df4.tail()

IDRSSD       AsOf                Name State     Cert   eTotAst  \
568382  5475452 2020-12-31  GULF ATLANTIC BANK    FL  59181.0   31303.0   
568383  5514409 2020-12-31         NELNET BANK    UT  59205.0  216939.0   
563335  5518023 2020-09-30       VARO BANK, NA    UT  59190.0  148876.0   
568384  5518023 2020-12-31       VARO BANK, NA    UT  59190.0  515114.0   
568385  5538937 2020-12-31   CLASSIC CITY BANK    GA  59219.0   38288.0   

         aTotAst   eTotLn     eDDep  eFDep   eTotDep  
568382   29355.0  14543.0   21478.0    0.0   21478.0  
568383  170333.0  17810.0  113046.0    0.0  113046.0  
563335  152132.0    840.0      38.0    0.0      38.0  
568384  464975.0   3564.0  415271.0    0.0  415271.0  
568385   21792.0  21302.0   19162.0    0.0   19162.0

In [11]:
df7 = df6x.merge(df4[['IDRSSD','AsOf','Name','State','Cert']],how='inner', left_on=['IDRSSD','AsOf'], right_on=['IDRSSD','AsOf'])
df7.columns = ['IDRSSD', 'xAsOf', 'xName', 'xState','xCert']
df7.head()

IDRSSD      xAsOf                             xName xState    xCert
0      37 2020-12-31            BANK OF HANCOCK COUNTY     GA  10057.0
1     242 2020-12-31  FIRST COMMUNITY BANK XENIA-FLORA     IL   3850.0
2     279 2020-12-31       MINEOLA COMMUNITY BANK, SSB     TX  28868.0
3     354 2020-12-31                  BISON STATE BANK     KS  14083.0
4     439 2007-06-30                      PEOPLES BANK     GA  16498.0

In [12]:
df7 = df7.merge(df6n[['IDRSSD','AsOf']],how='inner', left_on=['IDRSSD'], right_on=['IDRSSD'])
df7.columns = ['IDRSSD', 'xAsOf', 'xName', 'xState','xCert', 'nAsOf']
df7.tail()

IDRSSD      xAsOf               xName xState    xCert      nAsOf
10793  5472880 2020-12-31          CRAFT BANK     GA  59209.0 2020-12-31
10794  5475452 2020-12-31  GULF ATLANTIC BANK     FL  59181.0 2020-06-30
10795  5514409 2020-12-31         NELNET BANK     UT  59205.0 2020-12-31
10796  5518023 2020-12-31       VARO BANK, NA     UT  59190.0 2020-09-30
10797  5538937 2020-12-31   CLASSIC CITY BANK     GA  59219.0 2020-12-31

In [13]:
df5m = df5.mean()
df5m = df5m.reset_index()
df5m.drop('Cert', inplace=True, axis=1)
df8 = df7.merge(df5m, how='inner', left_on='IDRSSD', right_on='IDRSSD')

In [14]:
df8_orig = df8.copy()
len(df8) # 10762 (201612), 10767 (201712), 10775(201812), 10776(201903), 10787(201909), 10798(202012)

10798

In [15]:
df8.head()

IDRSSD      xAsOf                             xName xState    xCert  \
0      37 2020-12-31            BANK OF HANCOCK COUNTY     GA  10057.0   
1     242 2020-12-31  FIRST COMMUNITY BANK XENIA-FLORA     IL   3850.0   
2     279 2020-12-31       MINEOLA COMMUNITY BANK, SSB     TX  28868.0   
3     354 2020-12-31                  BISON STATE BANK     KS  14083.0   
4     439 2007-06-30                      PEOPLES BANK     GA  16498.0   

       nAsOf        eTotAst        aTotAst         eTotLn          eDDep  \
0 2001-03-31   80610.875000   80350.700000   37739.012500   64299.837500   
1 2001-03-31   35084.400000   35332.200000   21950.475000   26907.900000   
2 2001-03-31  166673.625000  161345.362500   88101.037500  118139.175000   
3 2001-03-31    9519.037500    9583.037500    4578.212500    8315.925000   
4 2001-03-31  166510.615385  165717.384615  118665.038462  144540.423077   

   eFDep        eTotDep  
0    0.0   64299.837500  
1    0.0   26907.900000  
2    0.0  118139.175000  
3    0.0    8315.925000  
4    0.0  144540.423077

In [16]:
df8.columns = ['IDRSSD','xAsOf','xName','xState','xCert','nAsOf','aeTotAst','aaTotAst','aeTotLn','aeDDep','aeFDep','aeTotDep']
df8.tail()

IDRSSD      xAsOf               xName xState    xCert      nAsOf  \
10793  5472880 2020-12-31          CRAFT BANK     GA  59209.0 2020-12-31   
10794  5475452 2020-12-31  GULF ATLANTIC BANK     FL  59181.0 2020-06-30   
10795  5514409 2020-12-31         NELNET BANK     UT  59205.0 2020-12-31   
10796  5518023 2020-12-31       VARO BANK, NA     UT  59190.0 2020-09-30   
10797  5538937 2020-12-31   CLASSIC CITY BANK     GA  59219.0 2020-12-31   

            aeTotAst       aaTotAst       aeTotLn         aeDDep  aeFDep  \
10793   24204.000000   21938.000000   1635.000000    3951.000000     0.0   
10794   23363.333333   20513.666667   8777.333333   12971.666667     0.0   
10795  216939.000000  170333.000000  17810.000000  113046.000000     0.0   
10796  331995.000000  308553.500000   2202.000000  207654.500000     0.0   
10797   38288.000000   21792.000000  21302.000000   19162.000000     0.0   

            aeTotDep  
10793    3951.000000  
10794   12971.666667  
10795  113046.000000  
10796  207654.500000  
10797   19162.000000

In [17]:
#############################
#############################
#  CAUTION: FailDate criterion here:
ff = pd.read_csv("CBfails_det202012.csv")

try:
  ## new colnames starting in 202003 csv file:
  ff2 = ff[['State', 'Year', 'NAME','CERT', 'FAILDATE']]  ## NOTE: Change in col names starting 202003
except:
  ## use next line (old colnames) for earlier versions of CB_fails_det201xxx.csv:
  ff2 = ff[['State', 'Year', 'Institution Name ','Cert ','Effective Date ']]

ff2.columns = ['State', 'Year', 'Name', 'Cert', 'EffDate']
ff2['FailDate'] = pd.to_datetime(ff2['EffDate'])
## ************************ ##
ff2 = ff2[ff2['FailDate']<='2020-12-31'] #  ONLY USE FAILDATES before cutoff date!!!!!!!!!!!! (to align with Call Data)
## ************************ ##
ff2.dropna(subset=['Cert'],inplace=True)
ff2['Cert'] = ff2['Cert'].astype(float)
ff2.head()

State    Year                              Name     Cert     EffDate  \
0    KS  2020.0                 ALMENA STATE BANK  15426.0  10/23/2020   
1    FL  2020.0        FIRST CITY BANK OF FLORIDA  16748.0  10/16/2020   
2    WV  2020.0              THE FIRST STATE BANK  14361.0    4/3/2020   
3    NE  2020.0                ERICSON STATE BANK  18265.0   2/14/2020   
4    NJ  2019.0  CITY NATIONAL BANK OF NEW JERSEY  21111.0   11/1/2019   

    FailDate  
0 2020-10-23  
1 2020-10-16  
2 2020-04-03  
3 2020-02-14  
4 2019-11-01

In [18]:
dff2 = ff2[ff2.duplicated(['Cert'])][['Name','Cert']]  ## find duplicate Certs
len(dff2)  # 44

44

In [19]:
ff2lst = list(dff2['Cert'])
len(ff2lst)                      ## list of dup Certs (28, now 44) (assistance, then failure)

44

In [20]:
ff2a = ff2[ff2.Cert.isin(ff2lst)]
ff2a.sort_values(['Cert','FailDate'],inplace=True)  ## sorted list of dup Certs
ff2a.head()

State    Year                              Name    Cert     EffDate  \
2382    TX  1988.0        FIRST CITY, TEXAS - DALLAS  1722.0   4/20/1988   
713     TX  1992.0    FIRST CITY, TEXAS - DALLAS, NA  1722.0  10/30/1992   
2383    TX  1988.0      FIRST CITY, TEXAS - BEAUMONT  3093.0   4/20/1988   
714     TX  1992.0  FIRST CITY, TEXAS - BEAUMONT, NA  3093.0  10/30/1992   
2381    TX  1988.0         FIRST CITY, TEXAS - BRYAN  3120.0   4/20/1988   

       FailDate  
2382 1988-04-20  
713  1992-10-30  
2383 1988-04-20  
714  1992-10-30  
2381 1988-04-20

In [21]:
## sections to dedup Cert field: (saves latest date, which usually is the fail date vs assistance date)

In [22]:
ff2x = ff2[['Cert','FailDate']].groupby('Cert').max()
ff2x.reset_index(drop=False,inplace=True)
ff2x['mx'] = 'x'
ff2x.tail()

Cert   FailDate mx
3567  90257.0 1992-08-21  x
3568  90266.0 1992-09-18  x
3569  90307.0 1992-08-14  x
3570  90310.0 1991-06-07  x
3571  91189.0 1998-07-23  x

In [23]:
ff3 = ff2.merge(ff2x,how='left',left_on=['Cert','FailDate'],right_on=['Cert','FailDate'])
ff3.tail()

State    Year                             Name     Cert    EffDate  \
3611    MO  1970.0  FIRST STATE BANK OF BONNE TERRE  12227.0  8/24/1970   
3612    NJ  1970.0          EATONTOWN NATIONAL BANK  19276.0   8/7/1970   
3613    KY  1970.0       FARMERS BANK OF PETERSBURG  13164.0  6/25/1970   
3614    MI  1970.0   THE PEOPLES STATE SAVINGS BANK  14987.0  4/18/1970   
3615    IA  1970.0       STATE BANK OF PRAIRIE CITY  14571.0  2/22/1970   

       FailDate mx  
3611 1970-08-24  x  
3612 1970-08-07  x  
3613 1970-06-25  x  
3614 1970-04-18  x  
3615 1970-02-22  x

In [24]:
ff4=ff3.dropna()
len(ff3), len(ff4) # 3616, 3572

(3616, 3572)

In [25]:
df8 = df8.merge(ff4[['Cert','FailDate']], how='left', left_on='xCert', right_on='Cert')
df8 = df8.drop('Cert', axis=1)
len(df8) # 10798

10798

In [26]:
df8.sort_values(['aeTotAst'],ascending=False, inplace=True)
df8.reset_index(drop=True, inplace=True)  # renumber rows based on largest to smallest banks
df8.reset_index(drop=False, inplace=True)  # move row numbers into another column called 'index'
df8.rename(columns={'index':'Rank'},inplace=True)  # rename 'index' column to 'Rank'
df8['Rank'] = df8['Rank'] + 1  # add 1 so that largest bank has rank 1 rather than rank 0
df8.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2020-12-31  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2020-12-31      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2020-12-31           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2020-12-31     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.593288e+09  1.567517e+09  5.652142e+08  7.671177e+08   
1 2001-03-31  1.341065e+09  1.339664e+09  6.952804e+08  8.832854e+08   
2 2001-03-31  1.111830e+09  1.108563e+09  5.233710e+08  3.160202e+08   
3 2001-03-31  9.768041e+08  9.652788e+08  5.772582e+08  6.953574e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate  
0  2.613865e+08  1.028504e+09        NaT  
1  1.040564e+08  9.873418e+08 2009-01-16  
2  4.547897e+08  7.708099e+08 2008-11-23  
3  6.515955e+07  7.605170e+08        NaT  
4  2.469325e+07  2.951088e+08        NaT

In [27]:
df8['pr_eTA'] = df8['aeTotAst']*100/df8['aeTotAst'].sum()
df8['pr_eTAcs'] = df8['pr_eTA'].cumsum()
df8['pr_eTL'] = df8['aeTotLn']*100/df8['aeTotLn'].sum()
df8['pr_eTLcs'] = df8['pr_eTL'].cumsum()
df8['pr_eDD'] = df8['aeDDep']*100/df8['aeTotLn'].sum()
df8['pr_eDDcs'] = df8['pr_eDD'].cumsum()
df8.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2020-12-31  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2020-12-31      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2020-12-31           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2020-12-31     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.593288e+09  1.567517e+09  5.652142e+08  7.671177e+08   
1 2001-03-31  1.341065e+09  1.339664e+09  6.952804e+08  8.832854e+08   
2 2001-03-31  1.111830e+09  1.108563e+09  5.233710e+08  3.160202e+08   
3 2001-03-31  9.768041e+08  9.652788e+08  5.772582e+08  6.953574e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate    pr_eTA   pr_eTAcs    pr_eTL  \
0  2.613865e+08  1.028504e+09        NaT  9.324842   9.324842  5.825125   
1  1.040564e+08  9.873418e+08 2009-01-16  7.848688  17.173530  7.165594   
2  4.547897e+08  7.708099e+08 2008-11-23  6.507073  23.680603  5.393887   
3  6.515955e+07  7.605170e+08        NaT  5.716821  29.397424  5.949251   
4  2.469325e+07  2.951088e+08        NaT  2.584770  31.982194  2.624301   

    pr_eTLcs    pr_eDD   pr_eDDcs  
0   5.825125  7.905952   7.905952  
1  12.990719  9.103182  17.009134  
2  18.384606  3.256919  20.266053  
3  24.333857  7.166387  27.432440  
4  26.958158  2.786916  30.219356

In [28]:
df8.to_csv("BkAvgAstPR_FailDate_202012.csv")

In [29]:
df8_fail = df8[df8['FailDate'].map(lambda x: x.year) > 2000]   #cbt[cbt['AsOf'].map(lambda x: x.month) == 12] 
len(df8_fail) # 521

521

In [30]:
df8_fail.to_csv("FailedBkList_AvgAst_202012.csv")